# Read All Dataset CSV

In [81]:
import os
import csv
import re
import pandas as pd
import numpy as np

In [82]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]

# 收集所有資料夾名稱，按照數字順序排序資料夾名稱
for folder_name in os.listdir("./Competition_data"):
    dataset_names.append(folder_name)
dataset_names = sorted(dataset_names, key=lambda x: int(re.search(r'\d+', x).group()))

for folder_name in dataset_names:
    # print(folder_name)
    X_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"./Competition_data/{folder_name}/X_test.csv",header=0))
    
    
    
# print(len(dataset_names))
# print(len(X_trains))  # 49, 代表有 49 個 dataFrame (每個資料集各一個)
# print(len(y_trains))
# print(len(X_tests))
# print(X_trains[0].dtypes)
# print(y_trains[0].dtypes)

In [83]:
# print(X_trains[42])

## Data Preprocessing & Feature Engineering

In [84]:
# from sklearn.preprocessing import OneHotEncoder
# from xgboost import XGBClassifier
# from pyod.models.iforest import IForest
# import pandas as pd
# import numpy as np

# for i in range(len(dataset_names)):
#     # Step 1: 分離數值型和類別型特徵
#     numerical_df = X_trains[i].select_dtypes(include=['float'])  # 數值型特徵
#     categorical_df = X_trains[i].select_dtypes(include=['int'])  # 類別型特徵

#     # Step 2: 使用 PYOD 的 IForest 檢測初始異常值
#     pyod_clf = IForest(contamination=0.1, random_state=42)  # 假設 10% 異常
#     pyod_clf.fit(numerical_df.values)
#     initial_outliers = pyod_clf.predict(numerical_df.values)  # 1 表示異常，0 表示正常

#     # Step 3: 將初始檢測結果作為 XGBoost 的訓練標籤
#     xgb_clf = XGBClassifier(
#         n_estimators=50,
#         random_state=42,
#         use_label_encoder=False,
#         eval_metric="logloss",
#     )
#     xgb_clf.fit(numerical_df, initial_outliers)

#     # Step 4: 使用 XGBoost 檢測異常值
#     final_outliers = xgb_clf.predict(numerical_df)

#     # Step 5: 過濾異常值
#     is_normal = (final_outliers == 0)  # XGBoost 中 0 表示正常
#     numerical_df = numerical_df[is_normal].reset_index(drop=True)
#     categorical_df = categorical_df[is_normal].reset_index(drop=True)
#     y_trains[i] = y_trains[i].iloc[is_normal].reset_index(drop=True)

#     # Step 6: 對類別型特徵進行 One-Hot 編碼
#     encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
#     categorical_encoded = encoder.fit_transform(categorical_df)
#     categorical_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out())

#     # 合併處理後的數據
#     X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)

#     # Step 7: 對測試數據進行相同處理
#     numerical_df_test = X_tests[i].select_dtypes(include=['float'])
#     categorical_df_test = X_tests[i].select_dtypes(include=['int'])

#     # 使用 XGBoost 預測測試數據中的異常值
#     test_outliers = xgb_clf.predict(numerical_df_test)
#     is_test_normal = (test_outliers == 0)
#     numerical_df_test = numerical_df_test[is_test_normal].reset_index(drop=True)
#     categorical_df_test = categorical_df_test[is_test_normal].reset_index(drop=True)

#     # 使用已訓練的 One-Hot Encoder 處理類別型特徵
#     categorical_encoded_test = encoder.transform(categorical_df_test)
#     categorical_df_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out())

#     # 合併測試數據
#     X_tests[i] = pd.concat([numerical_df_test, categorical_df_test], axis=1)

In [85]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer

# # 對每組資料進行處理
# for i in range(len(dataset_names)):
#     # 將連續型資料和數值型資料標準化
#     numerical_df = X_trains[i].select_dtypes(include=['float'])   # 數值型特徵
#     categorical_df = X_trains[i].select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）
#     if len(numerical_df.columns) + len(categorical_df.columns) != len(X_trains[i].columns):
#         print('Splitting error')
#     # numerical_df --> normalization
#     scaler = StandardScaler()
#     scaler.fit(numerical_df)
#     numerical_s = scaler.transform(numerical_df)
#     numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
#     X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)
    
    
#     numerical_df = X_tests[i].select_dtypes(include=['float'])   # 數值型特徵
#     categorical_df = X_tests[i].select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）
#     # 直接照前面用過的 scaler 來分
#     numerical_s = scaler.transform(numerical_df)
#     numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
#     X_tests[i] = pd.concat([numerical_df, categorical_df], axis=1)

# for i in range(len(dataset_names)):
#     y_trains[i].iloc[:, 0] = pd.to_numeric(y_trains[i].iloc[:, 0])

In [86]:
# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
# from sklearn.feature_selection import SelectKBest, chi2
# import pandas as pd

# # 对每组数据进行处理
# for i in range(len(dataset_names)):
#     print("before encoding: ", X_trains[i].shape[1])
#     # 将连续型数据和数值型数据标准化
#     numerical_df = X_trains[i].select_dtypes(include=['float'])   # 数值型特征
#     categorical_df = X_trains[i].select_dtypes(include=['int'])   # 类别型特征（可能为类别特征）
#     if len(numerical_df.columns) + len(categorical_df.columns) != len(X_trains[i].columns):
#         print('Splitting error')
    
#     # 数值型特征标准化
#     scaler = MinMaxScaler()
#     scaler.fit(numerical_df)
#     numerical_s = scaler.transform(numerical_df)
#     numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
    
#     # 类别型特征 One-Hot 编码
#     encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
#     categorical_s = encoder.fit_transform(categorical_df)
#     categorical_df = pd.DataFrame(categorical_s, columns=encoder.get_feature_names_out(categorical_df.columns))
    
#     # 合并处理后的数值型和类别型特征
#     X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)
    
#     # 处理测试数据
#     numerical_df_test = X_tests[i].select_dtypes(include=['float'])   # 数值型特征
#     categorical_df_test = X_tests[i].select_dtypes(include=['int'])   # 类别型特征（可能为类别特征）
    
#     # 使用相同的 scaler 对测试数据进行标准化
#     numerical_s_test = scaler.transform(numerical_df_test)
#     numerical_df_test = pd.DataFrame(numerical_s_test, columns=numerical_df_test.columns)
    
#     # 使用相同的 encoder 对测试数据进行 One-Hot 编码
#     categorical_s_test = encoder.transform(categorical_df_test)
#     categorical_df_test = pd.DataFrame(categorical_s_test, columns=encoder.get_feature_names_out(categorical_df_test.columns))
    
#     # 合并处理后的数值型和类别型特征
#     X_tests[i] = pd.concat([numerical_df_test, categorical_df_test], axis=1)

#     # 选择前 20 个重要特征
#     selector = SelectKBest(chi2, k=40)
#     X_trains[i] = selector.fit_transform(X_trains[i], y_trains[i])
#     X_tests[i] = selector.transform(X_tests[i])
#     print("after encoding: ", X_trains[i].shape[1])
# # 确保 y_trains 的标签转换为数值格式
# for i in range(len(dataset_names)):
#     y_trains[i].iloc[:, 0] = pd.to_numeric(y_trains[i].iloc[:, 0])

In [87]:
# for i in range(len(dataset_names)):
#     missing_cols = set(X_trains[i].columns) - set(X_tests[i].columns)
#     print(missing_cols)

## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [74]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## MODEL


In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

class YourTorchModel(nn.Module):
    def __init__(self,input_size):
        super(YourTorchModel, self).__init__()
        # 定義模型層
        self.layer0 = nn.Linear(input_size, 128)
        self.layer1 = nn.Linear(128, 32)
        # self.layer2 = nn.Linear(16, 12)
        # self.layer3 = nn.Linear(12, 8)
        self.out = nn.Linear(32, 1)

        self.act_fn = nn.ReLU()
        
        self.bn0 = nn.BatchNorm1d(128)
        self.bn1 = nn.BatchNorm1d(32)
        self.bn2 = nn.BatchNorm1d(12)
        self.bn3 = nn.BatchNorm1d(8)
        self.dropout = nn.Dropout(p=0.5)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):

        x = self.layer0(x)
        x = self.bn0(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        # x = self.layer2(x)
        # x = self.bn2(x)
        # x = self.act_fn(x)
        # x = self.dropout(x)

        # x = self.layer3(x)
        # x = self.bn3(x)
        # x = self.act_fn(x)
        # x = self.dropout(x)
        x = self.out(x)
        return torch.sigmoid(x)  # Apply sigmoid activation here




In [76]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Neuron network

In [77]:
# models = []
# avg_auc = 0
# # 根據不同數據續集訓練模型
# for i in range(len(dataset_names)):
#     # 使用 stratify 將數據分為訓練集和測試集
#     tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#         X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#     )
#     input_size = tmp_X_train.shape[1]
#     # 將數據轉換為 PyTorch 張量
#     # 將 DataFrame 轉為 numpy 陣列再轉為 PyTorch 張量
#     # 將數據轉換為 PyTorch 張量並移動到 CUDA (GPU)
#     tmp_X_train = torch.tensor(tmp_X_train.values, dtype=torch.float32).to(device)
#     tmp_y_train = torch.tensor(tmp_y_train.values, dtype=torch.float32).to(device)
#     tmp_X_test = torch.tensor(tmp_X_test.values, dtype=torch.float32).to(device)
#     tmp_y_test = torch.tensor(tmp_y_test.values, dtype=torch.float32).to(device)


#     # 初始化模型、損失函數和優化器
#     # model = YourTorchModel(input_size)      #if you don't have cuda gpu
#     model = YourTorchModel(input_size).to(device)
#     criterion = nn.BCELoss()  # 二元交叉熵損失
    
#     l2_lambda = 0.5  # L2 regularization factor
#     l1_lambda = 0.01  # L1 regularization factor   
#     optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)  

#     # 訓練迴圈
#     num_epochs = 300
    
#     train_acc = 0
    
#     for epoch in range(num_epochs):
#         model.train()
#         optimizer.zero_grad()
#         outputs = model(tmp_X_train)
#         loss = criterion(outputs, tmp_y_train)

#         # Apply L1 regularization
#         l1_penalty = sum(param.abs().sum() for param in model.parameters())
#         loss = loss + l1_lambda * l1_penalty

#         loss.backward()
#         optimizer.step()
#         #train acc
#         preds = (outputs > 0.5).float()  # 將概率轉換為類別標籤
#         train_acc += (preds == tmp_y_train).float().mean()  # 計算準確率

#     # 評估 - 預測測試集的概率
#     model.eval()
#     with torch.no_grad():
#         tmp_y_prob = model(tmp_X_test).squeeze().cpu().numpy() # 概率預測值
    
#     # 計算 AUC
#     # 經過資料處理後，可能會出現只有一個類別的情況，此時 AUC 會報錯，因此這裡做了一個判斷
#     unique_classes = np.unique(tmp_y_test.cpu().numpy())
#     if len(unique_classes) < 2:
#         print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
#         auc = 0.7  # 或選擇其他合適的值，例如預設值
#     else:
#         auc = roc_auc_score(tmp_y_test.cpu().numpy(), tmp_y_prob)

#     # print(f"{dataset_names[i]} AUC: {auc}")
#     avg_auc += auc
#     models.append(model)

#     train_acc /= num_epochs
#     # 在每個 dataset 結束時打印損失和準確率,and test auc
#     print(f"Dataset {dataset_names[i]}: Train Acc: {train_acc.item():.8f}, Test AUC: {auc:.8f}")   

# print("平均 AUC:", avg_auc / len(dataset_names))

### 把所有資料訓練集合後訓練(效果不是很好)

In [78]:
# models = []
# avg_auc = 0
# num_epochs = 100  # 訓練迴圈放到最外層
# # 把所有數據集放在一起訓練
# for epoch in range(num_epochs):
#     epoch_train_acc = 0
#     epoch_avg_auc = 0
#     print(f"Epoch {epoch + 1}/{num_epochs}")

#     for i in range(len(dataset_names)):
#         # 使用 stratify 將數據分為訓練集和測試集
#         tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#             X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#         )
#         input_size = tmp_X_train.shape[1]

#         # 將數據轉換為 PyTorch 張量並移動到 CUDA (GPU)
#         tmp_X_train = torch.tensor(tmp_X_train.values, dtype=torch.float32).to(device)
#         tmp_y_train = torch.tensor(tmp_y_train.values, dtype=torch.float32).to(device)
#         tmp_X_test = torch.tensor(tmp_X_test.values, dtype=torch.float32).to(device)
#         tmp_y_test = torch.tensor(tmp_y_test.values, dtype=torch.float32).to(device)

#         # 初始化模型、損失函數和優化器
#         # if epoch == 0 and i == 0:
#         model = YourTorchModel(input_size).to(device)
#         criterion = nn.BCELoss()  # 二元交叉熵損失
#         l2_lambda = 0.1
#         optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)

#         # 訓練
#         model.train()
#         optimizer.zero_grad()
#         outputs = model(tmp_X_train)
#         loss = criterion(outputs, tmp_y_train)
#         loss.backward()
#         optimizer.step()

#         # 計算訓練準確率
#         preds = (outputs > 0.5).float()
#         train_acc = (preds == tmp_y_train).float().mean()
#         epoch_train_acc += train_acc.item() / len(dataset_names)

#         # 評估 - 預測測試集的概率
#         model.eval()
#         with torch.no_grad():
#             tmp_y_prob = model(tmp_X_test).squeeze().cpu().numpy()

#         # 計算 AUC
#         auc = roc_auc_score(tmp_y_test.cpu().numpy(), tmp_y_prob)
#         epoch_avg_auc += auc / len(dataset_names)

#         # 保存模型（若在第一個 epoch 時，保存到模型列表中）
#         if epoch == 0:
#             models.append(model)

#     print(f"Epoch {epoch + 1}: Avg Train Acc: {epoch_train_acc:.8f}, Avg Test AUC: {epoch_avg_auc:.8f}")

# print("整體平均 AUC:", epoch_avg_auc)


## Cross vlidation(實驗中跑不動)


In [79]:
# from sklearn.model_selection import GridSearchCV, StratifiedKFold
# from sklearn.metrics import roc_auc_score

# # Define your parameter grid for XGBoost or any model you're tuning
# param_grid = {
#     'n_estimators': [100, 250, 300],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [3, 5, 7],
    
    
# }

# avg_auc = 0
# models = []

# for i in range(len(dataset_names)):
#     tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#         X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#     )
    
#     # Cross-validation with parameter tuning
#     model = XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='auc',
#     tree_method='gpu_hist',
#     use_label_encoder=False,
#     predictor='gpu_predictor'
#     )
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
#     grid_search.fit(tmp_X_train, tmp_y_train.squeeze())
    
#     # Get the best model from cross-validation
#     best_model = grid_search.best_estimator_
#     models.append(best_model)

#     # Evaluate the best model
#     tmp_y_prob = best_model.predict_proba(tmp_X_test)[:, 1]
#     unique_classes = np.unique(tmp_y_test)
#     if len(unique_classes) < 2:
#         print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
#         auc = 0.7
#     else:
#         auc = roc_auc_score(tmp_y_test, tmp_y_prob)
#     avg_auc += auc

#     print(f"AUC for dataset {i+1} with tuned parameters: {auc}")

# print(f"Average AUC across all datasets: {avg_auc / len(dataset_names)}")


## XGBoost

In [98]:
# 這邊用迴圈跑所有的資料集，並且將每個資料集的資料分成訓練集和測試集
# 並且用 Random Forest 來做分類
# 得到每個資料集的 AUC
# for max_deep in max_deeps:
max_deep  = 5
models=[]
avg_auc = 0
avg_train = 0
for i in range(len(dataset_names)):
    # 這邊做一下 stratify
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
    )

    # model = RandomForestClassifier(
    #     n_estimators=700, n_jobs=-1, random_state=2, max_depth=max_deep,  
    # )
    # XGBoost (好像比較容易過擬合，適合大資料集) 
   
    model = XGBClassifier(
        n_estimators=400, 
        eta=0.01,
        
        gamma=1, 
        max_depth=10, 
        subsample=0.7, 
        colsample_bytree=0.6,
        objective='binary:logistic',
        eval_metric='logloss',
        reg_lambda=1,     # L2 regularization term on weights
        alpha=0.1         # L1 regularization term on weights
    )
    model.fit(tmp_X_train, tmp_y_train.squeeze())
    
    unique_classes = np.unique(tmp_y_test)
    if len(unique_classes) < 2:
        print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
        auc = 0.7  # 或選擇其他合適的值，例如預設值
    else:
        tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    print(f'auc of dataset {i:2}: \t{auc}')
    avg_auc += auc
    models.append(model)
        
print(f"avg auc :   {avg_auc / len(dataset_names)}")
# print(f"avg auc of maxdeep of {max_deep}:   {avg_auc / len(dataset_names)}")



auc of dataset  0: 	0.8632478632478633
auc of dataset  1: 	1.0
auc of dataset  2: 	0.7755102040816326
auc of dataset  3: 	0.7577319587628866
auc of dataset  4: 	0.9713114754098361
auc of dataset  5: 	0.9957894736842104
auc of dataset  6: 	0.975
auc of dataset  7: 	0.8402777777777777
auc of dataset  8: 	0.8380952380952381
auc of dataset  9: 	0.7424242424242424
auc of dataset 10: 	0.7
auc of dataset 11: 	0.994949494949495
auc of dataset 12: 	0.8111888111888113
auc of dataset 13: 	1.0
auc of dataset 14: 	0.615
auc of dataset 15: 	1.0
auc of dataset 16: 	1.0
auc of dataset 17: 	0.9921875
auc of dataset 18: 	0.9850088183421516
auc of dataset 19: 	0.9791666666666667
auc of dataset 20: 	0.9857142857142858
auc of dataset 21: 	0.9375
auc of dataset 22: 	0.9865319865319865
auc of dataset 23: 	0.6383193277310923
auc of dataset 24: 	1.0
auc of dataset 25: 	0.8963210702341137
auc of dataset 26: 	0.9931350114416476
auc of dataset 27: 	0.8557291666666667
auc of dataset 28: 	0.8571428571428572
auc of 

## Inference Model

In [88]:
# #給neural network 用
# y_predicts = []
# for i in range(len(dataset_names)):
#     # Get the test data and convert to tensor if needed
#     X_test = torch.tensor(X_tests[i], dtype=torch.float32).to(device)  # Move to the same device as the model

#     # Put model in evaluation mode
#     models[i].eval()
    
#     # Disable gradient calculation for inference
#     with torch.no_grad():
#         # Pass data through the model and apply sigmoid to get probabilities
#         logits = models[i](X_test)  # logits will be on the same device as the model
#         y_predict_proba = torch.sigmoid(logits).cpu().numpy().flatten()  # Convert to numpy array

#     # Store the predictions as a DataFrame
#     df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
#     y_predicts.append(df)
#     #print auc
#     # print(f"Dataset {dataset_names[i]}: Test AUC: {roc_auc_score(y_trains[i], y_predict_proba)}")



In [92]:
##給xgboost用
y_predicts=[]
for i in range(len(dataset_names)):
    # print(X_tests[i])
    y_predict_proba=models[i].predict_proba(X_tests[i])[:, 1]
    df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
    y_predicts.append(df)
    

## Save result

In [93]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'./Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)

In [94]:
# from sklearn.metrics import roc_auc_score

# test_aucs = []
# for i in range(len(dataset_names)):
#     # 使用模型進行預測，獲得類別 1 的預測機率
#     y_predict_proba = models[i].predict_proba(X_tests[i])[:, 1]
    
#     # 計算 AUC 分數
#     auc = roc_auc_score(y_tests[i], y_predict_proba)
#     print(f'AUC of dataset {i:2}: \t{auc}')
    
#     test_aucs.append(auc)

# # 平均 AUC
# avg_test_auc = sum(test_aucs) / len(test_aucs)
# print("\nAverage Test AUC:", avg_test_auc)
